Tanner, Noah\
HW 0x03 Systems Modeling\
ME 405 - Professor Refvem\
Fall 2023

HW 0x03 Systems Modeling
ME 405 - Professor Refvem
Fall 2023

In [2]:
from numpy import array, arange, zeros, transpose
from matplotlib import pyplot, rc
from math import pi, cos, sin

Systems Model:\
\
In order to model the Romi robot system as a differential drive robot, and not a "unicycle" model, the independent characteristics of each wheel must be considered. In the image below, there is an independent velocity vector for the left and right wheels, as well as a combined velocity vector that makes up the velocity at a central point, inbetween the two wheels. The two coordinate systems, X,Y (Global), and X_R, Y_R (Local to the robot) are also pictured.
\
<img src="./Model_Pic.png" alt="Diagram of Romi Model" style="height:300px;" />
\
Some variables have been defined as well in order to understand the equations.
\
\
𝜔_r = Angular velocity of right wheel (rad/s)\
𝜔_l = Angular velocity of left wheel (rad/s)\
R       = Radius (R_r = R-l = R, unless otherwise noted)\
L       = Distance between the two wheels (Radius of rotation)

The following differential equations reflect the dynamics associated with a two wheel, differential drive robot. 

$$
\begin{array}{rl}
\frac{d}{dt}X           &= \left( \frac{R}{2} \left( \omega_r + \omega_l \right) 
\cos\theta \right)\\
\frac{d}{dt}Y           &= \left( \frac{R}{2} \left( \omega_r + \omega_l \right) 
\sin\theta \right)\\
\frac{d}{dt}\theta      &= \left( \frac{R}{L} \left( \omega_r + \omega_l \right) 
\right)
\end{array}
$$

Combining the preceding equations, a single vector equation can be obtained that can be solved using vector based ODE solution techniques.

$$
\begin{array}{rl}
\frac{d}{dt}
    \begin{bmatrix}
        X \\
        Y \\
        \theta
    \end{bmatrix}
&=
\begin{bmatrix}
    \frac{R}{2} \left( \omega_r - \omega_l \right)cos \theta \\
    \frac{R}{2} \left( \omega_r - \omega_l \right)sin \theta \\
    \frac{R}{L} \left( \omega_r - \omega_l \right)
\end{bmatrix} \\

\frac{d}{dt}
    \begin{bmatrix}
        X \\
        Y \\
        \theta
    \end{bmatrix}
&=
\begin{bmatrix}
    0 & 0 & 0 \\
    0 & 0 & 0 \\
    0 & 0 & 0 \\
\end{bmatrix}
    \begin{bmatrix}
        X \\
        Y \\
        \theta
    \end{bmatrix}
+
\begin{bmatrix}
    \frac{R}{L} cos\theta & \frac{R}{L} cos\theta \\
    \frac{R}{L} sin\theta & \frac{R}{L} sin\theta \\
    \frac{R}{L} & \frac{R}{L}
\end{bmatrix}
    \begin{bmatrix}
        \omega_r \\
        \omega_L
    \end{bmatrix}
\end{array}
$$



In [ ]:
# mechanical properties
R       = # radius of wheel [ in ]
L       = # distance between the two wheels [ in ]
theta   = # heading direction [ rad ]

# state - state couplin gmatrix
A = array([ [ 0, 0, 0 ],
            [ 0, 0, 0 ],
            [ 0, 0, 0 ] ])

# input = state coupling matrix
B = array([ [ (R/L)*cos(theta), (R/L)*cos(theta) ],
            [ (R/L)*sin(theta), (R/L)*sin(theta) ],
            [ R/L,              R/L              ] ])

# state - output coupling matrix
C = array([ [ 1, 0, 0, 0 ],
            [ 0, 1, 0, 0 ],
            [ 0, 0, 1, 0 ],
            [ 0, 0, 0, 1 ],
            [ 0, 0, 0, 0 ],
            [ 0, 0, 0, 0 ] ])

# input - output coupling matrix
D = array([ [ 0, 0 ],
            [ 0, 0 ],
            [ 0, 0 ],
            [ 0, 0 ],
            [ 1, 0 ],
            [ 0, 1 ] ])

In [ ]:
def system_eqn_ol(t, x):
    '''!@brief      Implements both state equations and output equations for the open loop system
        @param t    The value of time for a given simulation step
        @param x    The value of the state vector for a given simulation step
        @return     A tuple containing both the derivative of the state vector and the output
                    vector for a given simulation step
    '''
    
    # Constant input voltage for open-loop
    # The voltage must be packed in a 1x1 matrix for the arithmetic below
    u = array([ [12] ]);
    
    # State equations
    xd =  A@x+B@u;
    
    # Output Equations
    y  =  C@x+D@u;
    
    return xd, y

## Runge-Kutta Method (4th order)
The Runge-Kutta Method is the method of solving used in this homework as the integration technqiue is fourth order, allowing for a smaller truncation error for a given step size. This allows for a more reasonable step size while maintaining the reasonable accuracy of the solution. The algoritihm is similar to Euler's method, however it splits the window of time, $\Delta t$ in half, and computes the derivative several times at the start of the time window, in the middle, and at the end. These derivatives are used to find a weighted average which is the used in a standard Euler step. 
$$
\begin{array}{rl}
\mathbf{x}_{n+1} &= \mathbf{x}_{n} + \frac{1}{6} \left(\mathbf{k}_1 + 2 \mathbf{k}_2 + 2 \mathbf{k}_3 + \mathbf{k}_4 \right) \Delta t
\end{array}
$$
where,
$$
\begin{array}{rll}
\mathbf{k}_1 &= \mathbf{f}(t, & \mathbf{x}_n) \\
\mathbf{k}_2 &= \mathbf{f}(t+\frac{1}{2}\Delta t, & \mathbf{x}_n+\frac{1}{2} k_1 \Delta t) \\
\mathbf{k}_3 &= \mathbf{f}(t+\frac{1}{2}\Delta t, & \mathbf{x}_n+\frac{1}{2} k_2 \Delta t) \\
\mathbf{k}_4 &= \mathbf{f}(t+\Delta t, & \mathbf{x}_n+k_3 \Delta t).
\end{array}
$$

In [ ]:
def ruku4(fcn, x_0, tspan, tstep):
        '''!@brief        Implements a first-order forward euler solver
        @param fcn    A function handle to the function to solve
        @param x_0    The initial value of the state vector
        @param tspan  A span of time over which to solve the system specified as a list
                      with two elements representing initial and final time values
        @param tstep  The step size to use for the integration algorithm
        @return       A tuple containing both an array of time values and an array
                      of output values
    '''
    # define a column of time values
    tout = arange(tspan[0], tspan[1]+tstep, tstep)

    # preallocate an array of zeros to store state values
    xout = zeros([len(tout)+1,len(x_0)])

    # determine the dimension of the output vector
    r = len(fcn(0,x_0)[1])

    # preallocate an array of zeros to store output values
    yout = zeros([len(tout),r])

    # initialize output array with initial state vecotr
    xout[0][:] = x_0.T
    
    # iterate through algorithim, but stop one cycle early as the algorithim predicts one cycle ahead
    for n in range(len(tout)):
        
        # pull our a row from the solution array and transpose to get state vector as column
        x = xout[[n]].T

        # pull out the present value of time
        t = tout[n]

        # evaluate the f(n) handle at the preszent time with the present value of the state vector to compute the derivate
        k1, y = fcn( t, x )
        k2, y = fcn( t+.5*tstep, x+.5*k1*tstep )
        k3, y = fcn( t+.5*tstep, x+.5*k2*tstep )
        k4, y = fcn( t+tstep,    x+k3*tstep )
        
        # apply the update rule for RK4 method
        xout[n+1]   = xout[n] + (1/6)*(k1 + 2*k2 + 2*k3 + k4)*tstep
        yout[n]     = y.T

    return tout, yout

## Plot OL Results

In [ ]:
# Enlarge font size
rc('font', **{'size'   : 16})

pyplot.figure(figsize=(12,6))
pyplot.plot(t_OL, y_OL[:,1])
pyplot.xlabel('Time, t [s]')
pyplot.ylabel('Motor Velocity, [rad/s]')
pyplot.grid()